In [1]:
import os
import pandas as pd

from langchain_anthropic import ChatAnthropic

from logger import logger
from modules import extractor

C:\Users\Ohmatheus\AppData\Local\Temp\ipykernel_22108\4073229117.py:7: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from modules import extractor


to access modules in parent directory

In [2]:
import sys

# Add the parent directory (my_app/) to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

________________________________________________________

In [3]:
logger.info("-------------- New Notebook Session --------------")

2025-02-25 10:32:58,497 - LOGZ - INFO - -------------- New Notebook Session --------------


In [ ]:
with open("../api_key.txt") as f:
    api_key=f.read()

anthropic_model = "claude-3-7-sonnet-20250219"
logger.info(f"Using {anthropic_model} model.")

llm = ChatAnthropic(model=anthropic_model,
                    temperature=0,
                    max_tokens=512,
                    timeout=None,
                    max_retries=2,
                    api_key=api_key)

In [ ]:
def process_folder(folder_path, **kwargs):
    logger.info(f"Starting load and extraction of {folder_path} folder.")
    
    if not os.path.exists(folder_path):
        logger.error("Error: Folder {folder_path} does not exist.")
        return

    pdf_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

    if not pdf_files:
        logger.info("No PDF files found.")
        return

    logger.info(f"Found {len(pdf_files)} PDF(s). Processing...\n")

    results = []

    for pdf in pdf_files:
        pdf_path = os.path.join(folder_path, pdf)
        df = extractor.process_pdf(pdf_path, **kwargs)
        results.append(df)

    logger.info("Finished processing all PDFs.")
    return results

In [6]:
folder = '../Data/AllPDF'

args = {
    "llm": llm,
}

results = process_folder(folder, **args)

2025-02-25 10:32:58,530 - LOGZ - INFO - Found 9 PDF(s). Processing...

2025-02-25 10:32:58,531 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\ATX UMANUAL Draft 08-13 Edt 12-Rules app clean.pdf
2025-02-25 10:32:59,230 - LOGZ - INFO - Starting extraction on 1 chunks.
2025-02-25 10:33:03,462 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\Rules-and-Rating-Manual-3-1-20.pdf
2025-02-25 10:33:05,944 - LOGZ - INFO - Starting extraction on 14 chunks.
2025-02-25 10:33:17,354 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\Texas Accident Underwriting Guide - UPDATED MAY 2016 (clean).pdf
2025-02-25 10:33:17,479 - LOGZ - INFO - Starting extraction on 1 chunks.
2025-02-25 10:33:21,523 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\TPC Rule Manual.pdf
2025-02-25 10:33:21,909 - LOGZ - INFO - Starting extraction on 1 chunks.
2025-02-25 10:33:25,352 - LOGZ - INFO - Starting load and extraction of ../Data/AllPDF\TX AUTO AMENDATORY ENDORSEMENT

In [7]:
iterator = iter(results)
final_df = next(iterator)

try:
    while True:
        df = next(iterator)
        final_df = pd.concat([final_df, df], ignore_index=True)
except StopIteration:
    print("End of list reached")

final_df

End of list reached


,company_name,min_premium,min_premium_comments,policy_period,policy_period_comments,territory_factor_BI
0,Access General Agency Insurance Agency of Texa...,,No minimum premium information is provided in ...,1,"The document mentions ""one month policy"" in se...",
1,Texas Automobile Insurance Plan Association (T...,null,"The document mentions ""POLICY MINIMUM PREMIUM""...",6,While not explicitly stated in the provided te...,null
2,Texas Automobile Insurance Plan Association (T...,"$25 for Personal Auto Policies, $50 for all ot...",These are non-refundable minimum premiums for ...,12,The manual states that all rates and premiums ...,null
3,Texas Automobile Insurance Plan Association (T...,$20,"The document mentions ""$20 for the insured for...",6,The document defines a private passenger auto ...,null
4,Texas Automobile Insurance Plan Association (T...,,,36,"Based on the document, credits for driver impr...",
5,Texas Automobile Insurance Plan Association (T...,null,No minimum premium information is provided in ...,null,No policy period information is provided in th...,1.00
6,Texas Automobile Insurance Plan Association (T...,null,No minimum premium information is provided in ...,12,"The document mentions ""an auto hired, loaned, ...",$670
7,Texas Automobile Insurance Plan Association (T...,,,,,1.435
8,Texas Automobile Insurance Plan Association (T...,,,,,
9,Texas Automobile Insurance Plan Association (T...,,,,,


In [8]:
final_df.to_csv("extracted.csv", index=False)

how the hell am i going to handle tables ?